In [1]:
import bs4
import requests
from urllib.parse import unquote
from tqdm import tqdm
import time
from datetime import datetime, timezone
from feedgen.feed import FeedGenerator

In [2]:
course_page_request = requests.get('https://maktabkhooneh.org/course/%D8%A2%D8%B4%D9%86%D8%A7%DB%8C%DB%8C-%D8%A8%D8%A7-%D8%B1%DB%8C%D8%A7%D8%B6%DB%8C%D8%A7%D8%AA-mk99/')
course = bs4.BeautifulSoup(course_page_request.text, 'html.parser')

In [3]:
course_title = course.select_one('h1').text
course_description = course.select_one('.course-description').text

In [4]:
date_format = '%a, %d %b %Y %H:%M:%S %z'
pub_date = datetime.now().strftime(date_format)

In [5]:
fg = FeedGenerator()
fg.id(course.select_one('meta[property="og:url"]')['content'])
fg.title(course.select_one('meta[property="og:title"]')['content'])
fg.description(str(course.select_one('.course-description')))
fg.link(href=course.select_one('meta[property="og:url"]')['content'], rel='alternate')
fg.logo(course.select_one('meta[property="og:image"]')['content'])
#fg.link(href='https://publiclink/test.atom', rel='self')
fg.language('fa')

'fa'

In [6]:
i = 0

for chapter_link_element in course.select('a.chapter__unit'):
    chapter_link = chapter_link_element['href']
    r = requests.get('https://maktabkhooneh.org' + chapter_link)
    chapter = bs4.BeautifulSoup(r.content, 'html.parser')
    
    video_link = chapter.select_one('meta[property="og:video"]')['content'] # .replace('/videos/', '/videos/hq')
    video = requests.head(video_link)
    
    fe = fg.add_entry()
    fe.id('https://maktabkhooneh.org' + chapter_link)
    fe.title(chapter_link_element.select_one('.chapter__unit-title').text.strip())
    fe.description(chapter_link_element.select_one('.chapter__unit-title').text.strip())
    fe.enclosure(video_link, video.headers['Content-Length'], 'video/mp4')

    #fe.pubDate(video.headers['Last-Modified'])
    
    i += 1
    datetime_str = f'01/{str(i).zfill(2)}/20 00:00:00'
    datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
    datetime_object = datetime_object.replace(tzinfo=timezone.utc)
    fe.pubDate(datetime_object.strftime(date_format))

In [8]:
print(fg.rss_str(pretty=True).decode())

<?xml version='1.0' encoding='UTF-8'?>
<rss xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" version="2.0">
  <channel>
    <title>درس آشنایی با ریاضیات - مکتب خونه</title>
    <link>https://maktabkhooneh.org/course/آشنایی-با-ریاضیات-mk99/</link>
    <description>&lt;div class="course-description rich-text"&gt;
            در این با اصول ریاضیات و تاریخچه ی ریاضی آشنا میشویم . در هر جلسه از درس, استاد به معرفی هر یک از قسمت های علم ریاضی می پردازد و مقدمه ای بیان می کند. این درس به دوستان علاقه مند به ریاضیات پیشنهاد می شود. 
            
        &lt;/div&gt;</description>
    <docs>http://www.rssboard.org/rss-specification</docs>
    <generator>python-feedgen</generator>
    <image>
      <url>https://maktabkhooneh.org/media/courses/images/alishahi521385810120.png</url>
      <title>درس آشنایی با ریاضیات - مکتب خونه</title>
      <link>https://maktabkhooneh.org/course/آشنایی-با-ریاضیات-mk99/</link>
    </image>
    <language>fa</languag